
<div align="center">

<h1><font color='#fd7317'>Generación del modelo 182 a partir de los datos del fichero CSV de CALM</font></h1>
<h3><font color='#999999'>(para la FPV)</font></h3>
<img  width="100" src="https://www.hacienda.gob.es/es-ES/Prensa/En%20Portada/2020/PublishingImages/20181307_AEAT.jpg" />
</div>

In [3]:
#Code tells you how; Comments tell you why."
#CARGAR LIBRERIAS 
import pandas as pd
from matplotlib import pyplot as plt
#import io
#from google.colab import files
# filter_outliers
import numpy as np
from scipy import stats

#import warnings
#warnings.filterwarnings('ignore')

In [6]:
# LOAD CSV
df = pd.read_csv(r".\modelo182\Calm4AdHocReport_279_1579686916.csv")

,National Id,Family Name,Given Name,Tax State,Donation Amount,Currency
0,46450647P,Monge Aylagas,Sergio,Barcelona,200 €,EUR
1,73582690D,Moreno Piera,Fran,Valencia/València,220 €,EUR
2,00697079,Portilla,José Luis,Granada,100 €,EUR
3,46319749a,Navarro,Pere Josep,Tarragona,100 €,EUR
4,51080435B,Monsalve,Manuela,Alicante/Alacant,150 €,EUR


In [7]:
print("Número de estudiantes: " + str(len(df)))
print('Los nombres de las columnas son:\n \x1b[92m"' + '", "'.join([ col for col in df.columns]) + '"\x1b[0m')
npdf = df.values #convertir a numpy array


Número de estudiantes: 2744
Los nombres de las columnas son:
 "National Id", "Family Name", "Given Name", "Tax State", "Donation Amount", "Currency"


In [8]:
def filter_by(df, col_name):
  col_index = list(df.columns).index(col_name) #usamos posiciones, no nombres
  arr = df.values #convertir a numpy array
  filtered_entries = np.fromiter((x for x in arr if validate_DNI_NIE(x)), dtype=arr.dtype)
  left_out = np.fromiter((x for x in arr if not validate_DNI_NIE(x)), dtype=arr.dtype)

  positive_df = df[filtered_entries]
  negative_df = df[left_out]
  return (positive_df, negative_df)


In [9]:
#Valida el documento, tanto extranjero como nacional
#Devuelve True o False
def validate_DNI_NIE(dni):
    tabla = "TRWAGMYFPDXBNJZSQVHLCKE"
    dig_ext = "XYZ"
    reemp_dig_ext = {'X':'0', 'Y':'1', 'Z':'2'}
    numeros = "1234567890"
    dni = str(dni)
    dni = dni.upper()
    if len(dni) == 9:
        dig_control = dni[8]
        dni = dni[:8]
        if dni[0] in dig_ext:
            dni = dni.replace(dni[0], reemp_dig_ext[dni[0]])
        return len(dni) == len([n for n in dni if n in numeros]) \
            and tabla[int(dni)%23] == dig_control
    return False

In [12]:
# Validar el dni..
dni_mask = df["National Id"].apply(validate_DNI_NIE) 
# ..y separar el dataframe en dos grupos
dni_valido = df[dni_mask]
dni_no_valido = df[~dni_mask]

,National Id,Family Name,Given Name,Tax State,Donation Amount,Currency
2,00697079,Portilla,José Luis,Granada,100 €,EUR
6,H6354515,Valivarthi,Raju,Barcelona,120 €,EUR
7,11688406,Magno,Joana,Lisboa,100 €,EUR
16,1707ber00270,Faraus,Tim,Noord-Holland,150 €,EUR
18,23769808,Bagdiul,Ivona,Auvergne-Rhône-Alpes,1.000 €,EUR
...,...,...,...,...,...,...
2727,NaN,Martínez Fernàndez,Rosario,NaN,420 €,EUR
2733,NaN,Isaeva,Elizaveta,NaN,90 €,EUR
2734,Y6943691-M,Soto,Lucas,NaN,10 €,EUR
2737,501241388,Wilkinson,Richard,NaN,200 €,EUR
